In [5]:
from pickle import load
from numpy import argmax

from tensorflow.python.keras.models import load_model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras_preprocessing.image import load_img, img_to_array
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

In [2]:
curr_folder = "D:/YandexDisk/datasets/"
path_tokenizer = curr_folder + "ru-12k-tokenizer-train.pkl"
tokenizer = load(open(path_tokenizer, 'rb'))
max_words = 22

m0 = "D:/models/model-0.h5"
m1 = "D:/models/model-1.h5"
m2 = "D:/models/model-2.h5"
m3 = "D:/models/model-3.h5"
m4 = "D:/models/model-4.h5"
m5 = "D:/models/model-5.h5"
m6 = "D:/models/model-6.h5"
m7 = "D:/models/model-7.h5"
m8 = "D:/models/model-8.h5"
m9 = "D:/models/model-9.h5"
m10 = "D:/models/model-10.h5"
m11 = "D:/models/model-11.h5"
m12 = "D:/models/model-12.h5"
m13 = "D:/models/model-13.h5"
m14 = "D:/models/model-14.h5"
m15 = "D:/models/model-15.h5"
m16 = "D:/models/model-16.h5"
m17 = "D:/models/model-17.h5"
m18 = "D:/models/model-18.h5"
m19 = "D:/models/model-19.h5"

all_models = [m0, m1, m2, m3, m4,
              m5, m6, m7, m8, m9,
              m10, m11, m12, m13, m14,
              m15, m16, m17, m18, m19]

In [3]:
def extract_features(path_image):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    image = load_img(path_image, target_size=(224, 224))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)

    feature = model.predict(image, verbose=0)

    return feature

def map_int_to_word(integer, tokenizer):
    for word, idx in tokenizer.word_index.items():
        if idx == integer:
            return word

    return None

def generate_caption(model, tokenizer, max_words, image):
    in_text = 'startseq'

    for i in range(max_words):
        seq = tokenizer.texts_to_sequences([in_text])[0]
        seq = pad_sequences([seq], maxlen=max_words)

        y_hat = model.predict([image,seq], verbose=0)
        y_hat = argmax(y_hat)

        word = map_int_to_word(y_hat, tokenizer)
        if word is None:
            break

        in_text += ' ' + word

        if word == 'endseq':
            break

    return in_text

def print_image(path_image):
    image = mpimg.imread(path_image)
    plt.imshow(image)
    plt.show()

def print_image_with_caption(list_path_model, path_image):
    print_image(path_image)
    image = extract_features(path_image)

    for i, path in enumerate(list_path_model):
        model = load_model(path)
        caption = generate_caption(model, tokenizer, max_words, image)
        print(str(i) + ' ' + caption)